# MotionSSM

## 1 Create Datasets

### 1.1 Download Data

In [32]:
import os
from huggingface_hub import hf_hub_download


## here download, unzip the data
if os.path.exists('./DanceTrack/train'):
    print('Data Already Downloaded!')
else:
    repo_id = 'noahcao/dancetrack'
    filename = 'train1.zip'
    save_dir = './DanceTrack'
    hf_hub_download(repo_id ,filename, repo_type='dataset',local_dir=save_dir)
    
    filename = 'train2.zip'
    hf_hub_download(repo_id ,filename, repo_type='dataset',local_dir=save_dir)

## here download, unzip the data
if os.path.exists('./DanceTrack/val'):
    print('Data Already Downloaded!')
else:
    repo_id = 'noahcao/dancetrack'
    filename = 'val.zip'
    save_dir = './DanceTrack'
    
    hf_hub_download(repo_id ,filename, repo_type='dataset',local_dir=save_dir)

train1.zip:   0%|          | 0.00/3.61G [00:00<?, ?B/s]

### 1.2 Unzip Data 

In [33]:
import zipfile
zip_paths = ['train1.zip', 'train2.zip', 'val.zip']
tgt_paths = ['train','train','val']

for i in range(len(zip_paths)):
    with zipfile.ZipFile('./DanceTrack/' + zip_paths[i], 'r') as zip_ref:
        zip_ref.extractall('./DanceTrack/' + tgt_paths[i])
    os.remove('./DanceTrack/' + zip_paths[i])
print('Extracted Data And Removed Zip Files.')

Extracted data and removed zip files.


### 1.3 Restructure Directory

In [36]:
from pathlib import Path
import shutil

paths = ['train/train1','train/train2', 'val/val']
for path in paths:
    src_path = Path('./DanceTrack').joinpath(path)
    tgt_path = src_path.parent

    for src_file in src_path.glob('*dancetrack*'):
        shutil.move(src_file, tgt_path)
    shutil.rmtree('./DanceTrack/' + path)
print('Restructured Directory Successfully.')

FileNotFoundError: [Errno 2] No such file or directory: './DanceTrack/train/train1'

### 1.4 Download Seqmap Files

In [37]:
from urllib.request import urlretrieve
url = 'https://raw.githubusercontent.com/DanceTrack/DanceTrack/refs/heads/main/dancetrack/'
seqmap_files = ['train_seqmap.txt', 'val_seqmap.txt']

for file in seqmap_files:
    urlretrieve(url + file, './DanceTrack/' + file)


### 1.5 Preprocess Train Data

We are changing the ground-truth format from '[bb_left, bb_top, bb_width, bb_height]' to '[bb_center_x, bb_center_y, bb_width, bb_height]'. Moreover, the ground-truth data is split into seperate sequences for each tracked object.

In [39]:
from preprocess_data import preprocess_train

seq_root = './DanceTrack'
label_root = './DanceTrack/train_seq'
preprocess_train(seq_root, label_root)

dancetrack0008
dancetrack0015
dancetrack0029
dancetrack0032
dancetrack0001
dancetrack0072
dancetrack0052
dancetrack0012
dancetrack0080
dancetrack0062
dancetrack0099
dancetrack0051
dancetrack0033
dancetrack0027
dancetrack0061
dancetrack0082
dancetrack0044
dancetrack0086
dancetrack0069
dancetrack0020
dancetrack0096
dancetrack0075
dancetrack0002
dancetrack0006
dancetrack0045
dancetrack0039
dancetrack0053
dancetrack0074
dancetrack0023
dancetrack0024
dancetrack0098
dancetrack0049
dancetrack0037
dancetrack0057
dancetrack0066
dancetrack0083
dancetrack0087
dancetrack0016
dancetrack0068
dancetrack0055


### 1.5 Download Validation Yolo-X Detections

In [57]:
if os.path.exists('./DanceTrack/val_dets'):
    print('Data Already Downloaded!')
else:
    urlretrieve('https://github.com/Kroery/DiffMOT/releases/download/v1.1/Detections.zip', './DanceTrack/Detections.zip')
    with zipfile.ZipFile('./DanceTrack/Detections.zip', 'r') as zip_ref:
        zip_ref.extractall('./DanceTrack/Detections')
    os.remove('./DanceTrack/Detections.zip')
    
    src_path = Path('./DanceTrack/Detections/DanceTrack/detections_yolox_x/val')
    os.mkdir('./DanceTrack/val_dets')
    tgt_path = './DanceTrack/val_dets'
    for src_file in src_path.glob('*dancetrack*'):
        shutil.move(src_file, tgt_path)


DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0030
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0058
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0065
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0073
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0094
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0081
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0077
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0014
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0097
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0034
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0043
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0047
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0007
DanceTrack/Detections/DanceTrack/detections_yolox_x/val/dancetrack0018
DanceT

## 1.6 Build Dataset

In [1]:
from dataset import MotionDataset

seq_len = 5
data_dir = './DanceTrack/train_seq'
dataset_train = MotionDataset(seq_len, data_dir)
print(dataset_train.__getitem__(4))

{'gt': array([ 0.      , 11.      ,  0.254297,  0.646528,  0.075781,  0.281944,
        1.      ], dtype=float32), 'gt_box': array([0.254297, 0.646528, 0.075781, 0.281944], dtype=float32), 'seq_features': tensor([[ 0.2570,  0.6500,  0.0797,  0.2750,  0.0027,  0.0035,  0.0039, -0.0069],
        [ 0.2633,  0.6493,  0.0891,  0.2736,  0.0062, -0.0007,  0.0094, -0.0014],
        [ 0.2672,  0.6521,  0.0953,  0.2681,  0.0039,  0.0028,  0.0062, -0.0056],
        [ 0.2688,  0.6535,  0.1016,  0.2625,  0.0016,  0.0014,  0.0063, -0.0056],
        [ 0.2660,  0.6549,  0.0977,  0.2597, -0.0027,  0.0014, -0.0039, -0.0028]])}


/home/fredo/PycharmProjects/MotionSSM/dataset.py:70: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  seq_features = torch.concatenate([torch.tensor(seq_boxes[1:]), torch.tensor(delta_boxes)], dim=1)


## 2 Build Model

In [2]:
from model.motion_ssm import MotionSSM
config = ''
model = MotionSSM(config)
x = dataset_train.__getitem__(4)
print(x['seq_features'].unsqueeze(0).shape)
x['seq_features'] = x['seq_features'].unsqueeze(0)
model.train()
model.forward(x)

torch.Size([1, 5, 8])


NameError: name 'causal_conv1d_fn' is not defined

## 3 Train Model

## 4 Evaluate Model